In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

## Get subregional simulation output

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\n   Max run id : {:,}".format(run_id))

hs_change_sql = '''
    SELECT o.parcel_id, j.name,  p.cap_jurisdiction_id, p.jurisdiction_id, p.mgra_id, p.luz_id,
    unit_change as hs_change, source, year_simulation
      FROM urbansim.urbansim.urbansim_lite_output o 
      JOIN urbansim.urbansim.parcel p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id
     WHERE run_id =  %s
  ORDER BY j.name,p.jurisdiction_id, year_simulation'''
hs_change_sql = hs_change_sql % run_id
hs = pd.read_sql(hs_change_sql,mssql_engine)
print("\n   Units added: {:,}".format(int(hs.hs_change.sum())))

## QC check output against jurisdiction feedback confluence pg (since all capacity used)

https://sandag.atlassian.net/wiki/spaces/LUM/pages/101679105/Jurisdictional+Feedback

In [ ]:
jur_cap_df = pd.DataFrame({'units_change': hs.groupby(['cap_jurisdiction_id']).hs_change.sum()}).reset_index()
jur_cap_df.set_index('cap_jurisdiction_id',inplace=True)
jur_cap_df

## Get total dwelling units in the region and sum by jurisdiction and cpa

#### note using cap jurisdiction id

In [ ]:
du_sql = '''
    SELECT parcel_id, mgra_id, luz_id, p.jurisdiction_id, cap_jurisdiction_id, j.name, du, capacity 
        FROM urbansim.parcel p
        LEFT JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id
        WHERE du > 0'''
du = pd.read_sql(du_sql,mssql_engine)
du.cap_jurisdiction_id.fillna(du.jurisdiction_id,inplace=True) # where there is no cap jurisdiction id 
print("\n   Dwelling Units: {:,}".format(int(du.du.sum())))

## Get CPAs for city and county

#### complete list for plotting, i.e. plot zero for those with no change

In [ ]:
xref_geography_sql = '''
    SELECT mgra_13, cocpa_2016, cicpa_13
      FROM data_cafe.ref.vi_xref_geography_mgra_13
      where jurisdiction_2016 IN (14,19)'''
xref_geography_df = pd.read_sql(xref_geography_sql, mssql_engine)
# remove mgras without a CPA (mgra_13 = 7259)
xref_geography_df = xref_geography_df.loc[~((xref_geography_df.cocpa_2016.isnull()) & (xref_geography_df.cicpa_13.isnull()))].copy()
len(xref_geography_df)
print("\n Note: {:,} MGRAs in City and County CPAs (of 23,002 MGRAs)".format(len(xref_geography_df)))

#### for jurisdiction 19 use cocpa_2016 and for jurisdiction 14 use cicpa_13

In [ ]:
# simulation output
units = pd.merge(hs,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how = 'outer')
units['jcid'] = units['cap_jurisdiction_id']
units.loc[units.cap_jurisdiction_id == 19,'jcid'] = units['cocpa_2016']
units.loc[units.cap_jurisdiction_id == 14,'jcid'] = units['cicpa_13']
# dwelling units
dus = pd.merge(du,xref_geography_df,left_on='mgra_id',right_on='mgra_13',how = 'outer')
dus['jcid'] = dus['cap_jurisdiction_id']
dus.loc[dus.cap_jurisdiction_id == 19,'jcid'] = dus['cocpa_2016']
dus.loc[dus.cap_jurisdiction_id == 14,'jcid'] = dus['cicpa_13']

In [ ]:
len(xref_geography_df)

In [ ]:
len(hs)

In [ ]:
len(units)

### In cases where there was no unit change for an MGRA, fill in with zero values

#### note: joined on mgra for geography

In [ ]:
# CPAs = units.jcid.unique().tolist() # CPAS in df
# units.loc[(units.mgra_id.isnull()) & (~units.cicpa_13.isin(CPAs))]

In [ ]:
units.loc[units['hs_change'].isnull()].head(2)

In [ ]:
units.loc[units.jcid.isnull(),'jcid'] = units['cicpa_13']
units.loc[units.jcid.isnull(),'jcid'] = units['cocpa_2016']
units.fillna(0, inplace=True)
dus.loc[dus.jcid.isnull(),'jcid'] = dus['cicpa_13']
dus.loc[dus.jcid.isnull(),'jcid'] = dus['cocpa_2016']
dus.fillna(0, inplace=True)

In [ ]:
units.loc[units['hs_change']==0].head(2)

### In cases where a parcel has no CPA but in jurisdiction 14 or 19, find nearest CPA for plotting

In [ ]:
units.loc[units['jcid']==0]

In [ ]:
dus.loc[dus['jcid']==0]

## use sql query to find nearest CPA

#### set jcid to nearest CPA and change cocpa_2016 or cicpa_13 to match (for looking up name later)

In [ ]:
# for plotting: change mgra 19415 to 19423 in county bc it has capacity from the county but is in San Marcos
# mgra=19415 is in Ciyt=15 San Marcos but has capacity of 5 from County so assigning it a nearby mgra
# so it will now be CPA 1909 (for the purposes of plotting)
units.loc[units.mgra_id==19415,'jcid'] = 1909
units.loc[units.mgra_id==19415,'cocpa_2016'] = 1909
# unit.loc[units.mgra_id == 19415,'mgra_id'] = 19423

In [ ]:
dus.loc[dus.mgra_id==18831.0,'jcid'] = 1909
dus.loc[dus.mgra_id==18831.0,'cocpa_2016'] = 1909
dus.loc[dus.mgra_id==11514.0,'jcid'] = 1914
dus.loc[dus.mgra_id==11514.0,'cocpa_2016'] = 1914
dus.loc[dus.mgra_id==7521.0,'jcid'] = 3 # North Island not jurisdiction 14?
dus.loc[dus.mgra_id==19415.0,'jcid'] = 1909
dus.loc[dus.mgra_id==19415.0,'cocpa_2016'] = 1909

#### convert jcid column that has jurisdiction id and CPAs to integer (only possible if no null values in column)

In [ ]:
units['jcid'] = units['jcid'].astype(int) 
dus['jcid'] = dus['jcid'].astype(int)

### Add cpa name

In [ ]:
cocpa_names_sql = '''
    SELECT zone as cocpa_id, name as cocpa
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 20'''
cocpa_names = pd.read_sql(cocpa_names_sql, mssql_engine)
cicpa_names_sql = '''
    SELECT zone as cicpa_id, name as cicpa
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 15'''
cicpa_names = pd.read_sql(cicpa_names_sql, mssql_engine)
luz_names_sql = '''
    SELECT zone as luz_id, name as luz
    FROM data_cafe.ref.geography_zone WHERE geography_type_id = 64'''
luz_names = pd.read_sql(luz_names_sql, mssql_engine)     

In [ ]:
units = pd.merge(units,cocpa_names,left_on='cocpa_2016',right_on='cocpa_id',how = 'left')
units = pd.merge(units,cicpa_names,left_on='cicpa_13',right_on='cicpa_id',how = 'left')

In [ ]:
units['jcname'] = units['name']
units.loc[units.jcid>=1900,'jcname'] = units['cocpa']
units.loc[(units.jcid>=1400) & (units.jcid<1900),'jcname'] = units['cicpa']

In [ ]:
# check that there are no nulls values (should equal 103)
len(units.jcname.unique())

#### check results again to match jurisdiction feedback page

In [ ]:
change_df = pd.DataFrame({'chg': units.groupby(['jcid','jcname']).
                               hs_change.sum()}).reset_index()
change_df['jcid'] =change_df['jcid'].astype(int)
change_df.set_index('jcid',inplace=True)
# change_df

## sum dwelling units by jursidictions and CPAs (n=103)

In [ ]:
du_sr14_geo_df = pd.DataFrame(dus.groupby(['jcid']).\
                            du.sum()).reset_index()
du_sr14_geo_df.sort_values(by='jcid',inplace=True)
du_sr14_geo_df.set_index('jcid',inplace=True)
du_sr14_geo_df['du'] = du_sr14_geo_df['du'].astype(int)
print("\n Total residential dwelling units after groupby: {:,}".format(int(du_sr14_geo_df.du.sum())))
print("\n Total number of jurisdictions and cpas: {:,}\n".format(len(du_sr14_geo_df.index.unique())))

## sum hs change in simulation by jursidictions and CPAs (n=103)

In [ ]:
sr14_geo_df = pd.DataFrame({'hs_sum': units.groupby(['jcid','jcname','year_simulation']).\
                            hs_change.sum()}).reset_index()
sr14_geo_df.rename(columns = {'jcname':'geo'},inplace=True)
# sr14_geo_df.rename(columns = {'year_simulation':'increment'},inplace=True)
sr14_geo_df.sort_values(by='jcid',inplace=True)
sr14_geo_df.set_index('jcid',inplace=True)
sr14_geo_df['hs_sum'] = sr14_geo_df['hs_sum'].astype(int)
sr14_geo_df['year_simulation'] = sr14_geo_df['year_simulation'].astype(int)
print("\n Total housing unit change after groupby: {:,}".format(int(sr14_geo_df.hs_sum.sum())))
print("\n Total number of jurisdictions and cpas: {:,}\n".format(len(sr14_geo_df.index.unique())))

## Fill in "0" for units for "missing" simulation years (for plotting) (e.g. Del Mar)

In [ ]:
# Del Mar example
del_mar_before = sr14_geo_df.loc[4].sort_values(by='year_simulation')
# del_mar_before.head()
del_mar_before.plot(x='year_simulation',y='hs_sum',style='.-',title='NULL values in Del Mar Housing Unit Change')

In [ ]:
idx = range(2017,2051)
sr14_geo_df.set_index(['geo','year_simulation'],append=True,inplace=True)
sr14_geo_df = sr14_geo_df.unstack(['jcid','geo'])
sr14_geo_df = sr14_geo_df.reindex(idx, fill_value=0)
sr14_geo_df.fillna(0,inplace=True)
sr14_geo_df = sr14_geo_df.stack(['jcid','geo'])
sr14_geo_df.reset_index(inplace=True)
sr14_geo_df.set_index('jcid',inplace=True)

In [ ]:
del_mar_after = sr14_geo_df.loc[4].sort_values(by='year_simulation')
del_mar_after.plot(x='year_simulation',y='hs_sum',style='.-',title='Replace Null with Zeroes Del Mar Housing Unit Change')

In [ ]:
len(sr14_geo_df.geo.unique())

In [ ]:
len(du_sr14_geo_df)

In [ ]:
len(sr14_geo_df.year_simulation.unique())

In [ ]:
len(sr14_geo_df.year_simulation.unique()) * len(sr14_geo_df.geo.unique())

In [ ]:
len(sr14_geo_df)

## Sum units from output of simulation over five year increments

In [ ]:
bins = range(2015,2055,5)
names = [str(x) for x in range(2020,2055,5)]
sr14_geo_df['increment'] = pd.cut(sr14_geo_df.year_simulation, bins, labels=names)

In [ ]:
sr14_increment = pd.DataFrame({'hs_increment': sr14_geo_df.
                                            groupby(["increment","jcid","geo"]).
                                 hs_sum.sum()}).reset_index()
# sr14_increment.set_index('jcid',inplace=True)

In [ ]:
sr14_increment.head()

## Cumulative sum units added by increment

In [ ]:
sr14_increment['hs_cumulative'] = sr14_increment.groupby(['geo'])['hs_increment'].apply(lambda x: x.cumsum())
sr14_increment.set_index('jcid',inplace=True)

In [ ]:
sr14_increment.loc[sr14_increment.geo=='Carlsbad']

## Add increment 2016 with units added equal to zero for baseline du (for plotting)¶

In [ ]:
start_year = sr14_geo_df.loc[sr14_geo_df.year_simulation==2017].copy()

In [ ]:
len(start_year)

In [ ]:
start_year['increment'] = '2016'
start_year['hs_cumulative'] = 0
start_year['year_simulation'] = 'baseline'

In [ ]:
sr14_increment = pd.concat([sr14_increment,start_year])

## Join simulation output with existing dwelling units

In [ ]:
len(sr14_increment)

In [ ]:
len(du_sr14_geo_df)

In [ ]:
sr14 = sr14_increment.join(du_sr14_geo_df)

In [ ]:
sr14['hs'] = sr14['hs_cumulative'] + sr14['du']

In [ ]:
sr14.head()

# plot

In [ ]:
sr14_geo_df_pivot = sr14.pivot\
(index='increment', columns='geo', values='hs').\
reset_index().rename_axis(None, axis=1)
sr14_geo_df_pivot.set_index('increment',inplace=True)
sr14_geo_df_pivot.fillna(0,inplace=True)
# sr14_geo_df_pivot.head(35)

In [ ]:
sr14_geo_df_pivot.head()

In [ ]:
pp = PdfPages("jur_and_cpa_sr14.pdf")
for j, jur in enumerate(sr14_geo_df.geo.unique().tolist()):
    chg = int(sr14_increment.loc[(sr14_increment.increment=='2050') & (sr14_increment.geo==jur)].hs_cumulative)
    jur_and_cpa_plot = plt.figure()
    # plt.subplot(20, 1, j+1)
    # plotlabel = jur + '\nchg = ' + str(int(totalchange.loc[jur][0]))
    plotlabel = str(sr14_geo_df.loc[sr14_geo_df['geo']==jur].index.values[0]) + '.' +\
                jur + '\nchg = ' + str(chg)
    plt.plot(sr14_geo_df_pivot[[jur]].reset_index().increment.tolist(),\
             sr14_geo_df_pivot[[jur]].reset_index()[jur].tolist(),\
             label = plotlabel,marker='o')
    plt.legend()
    plt.ylabel('Housing stock')
    plt.xlabel('Increment')
    plt.title('Series14')
    pp.savefig(jur_and_cpa_plot, dpi = 300, transparent = True)
pp.close()
# plt.savefig('sr13_jur_and_cpa.png')